# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [1]:
import argparse
import os
import re
import requests


def wget(url):
    """
      Function for requesting urls
      args:
        The URL to be requested
      response:
        The content of the response
    """
    def handleLoadToVar(resp):
      """
        Function for chunking the http response in order to manage system resources
        args:
          resp: The http response
        response:
          The chunked response
      """
      result = bytearray()
      resp.raise_for_status()      
      for chunk in resp.iter_content(chunk_size=8192): 
        result += chunk
      return result
    
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True, stream = True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    return handleLoadToVar(resp)

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [2]:
import requests
from urllib.parse import quote
import hashlib

class Document:
    
    def __init__(self, url):
        self.url = url
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):      
        #TODO download self.url content, store it in self.content and return True in case of success
        try:  
          result = wget(self.url)  
          if(result is None):
            return False
          self.content = result
          return True
        except:
          return False
    
    def persist(self):
        #We use the hash of the url to save the file
        filename = hashlib.sha256(self.url.encode()).hexdigest()
        with open(filename, "wb") as file:
          file.write(self.content)
            
    def load(self):  
        #TODO load content from hard drive, store it in self.content and return True in case of success    
        try:
          filename = hashlib.sha256(self.url.encode()).hexdigest()
          with open(filename, "rb") as file:
            self.content = file.read()
          return True
        except:
          return False

### 1.1.1. Tests ###

In [3]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

X-Powered-By: Express
Accept-Ranges: bytes
Cache-Control: public, max-age=0
Last-Modified: Mon, 23 Apr 2018 08:15:13 GMT
ETag: W/"17eb-162f1923268"
Content-Type: text/plain; charset=UTF-8
Content-Length: 6123
Date: Sun, 12 Feb 2023 09:51:53 GMT
Connection: keep-alive



## 1.2. [10] Parse HTML
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
1. `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
2. `self.images` list of images met in a document. Again, links can be relative to current page.
3. `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [4]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import re

class HtmlDocument(Document):
    #TODO extract plain text, images and links from the document    

    def parse(self):
        def _preprocess_link(link):
          """
            Function for handling relative urls
            args:
              link: The urls to be processed
            response:
              The processed urls
          """
          if re.match("(?:^[a-z][a-z0-9+\.-]*:|\/\/)",link ):
            return link
          else:
            return urllib.parse.urljoin(self.url,link)

        def _get_anchors(dom):
          """
            Function for getting the urls and the corresponding tags in the dom
            args:
              dom: The dom 
            response:
              A list of all anchor links and names              
          """
          all_hrefs = dom.find_all('a', href=True)
          all_urls = set()
          return list(set((a.text,_preprocess_link(a['href'])) for a in all_hrefs))
        
        def _get_images(dom):
          """
            Function for getting the urls and the corresponding tags in the dom
            args:
              dom: The dom 
            response:
              A list of all the image sources              
          """          
          all_images= dom.find_all('img', src=True)
          all_src = set()          
          return list(set([_preprocess_link(img['src']) for img in all_images]))                  
        
        def tag_visible(element):
          """
            Function for checking if a html element is among the visible elements in the dom
            args:
              element: The HTML element to be checked
            response: A boolean specifying if an element is visible                                            
          """
          if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
              return False
          if isinstance(element, Comment):
              return False
          return True
        
        def _get_text(dom):
          """
            Function for getting visible texts in the dom
            args:
              dom: The dom 
            response: A string of all the visible texts             
          """
          texts = dom.findAll(text=True)
          visible_texts = filter(tag_visible, texts)
          return u" ".join(t.strip() for t in visible_texts)
        
        try:
          dom = BeautifulSoup(self.content.decode())  
          self.anchors = _get_anchors(dom)
          self.images = _get_images(dom)
          self.text = _get_text(dom)
        except Exception as e:
          print(e)
          pass

### 1.2.1. Tests

In [5]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

X-Powered-By: Express
Accept-Ranges: bytes
Cache-Control: public, max-age=0
Last-Modified: Wed, 13 Apr 2022 14:34:29 GMT
ETag: W/"146f-1802358e108"
Content-Type: text/html; charset=UTF-8
Content-Length: 5231
Date: Sun, 12 Feb 2023 09:51:53 GMT
Connection: keep-alive



## 1.3. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria to succeed in the task**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained from inside `<body>` tag only.

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from collections import Counter
from nltk import tokenize

class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        try:
          self.doc.get()
          self.doc.parse()
        except FileNotFoundError:
          print("File Not Found")
    
    def get_sentences(self):
        #TODO implement sentence parser
        result = nltk.sent_tokenize(self.doc.text.strip())
        return result
    
    def get_word_stats(self):
        result = nltk.word_tokenize(self.doc.text.strip())
        #TODO return Counter object of the document, containing mapping {`word` -> count_in_doc}
        return Counter(map(str.lower,result))

### 1.3.1. Tests ###

In [8]:
doc = HtmlDocumentTextData("https://innopolis.university/")
print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

Server: nginx/1.22.0
Date: Sun, 12 Feb 2023 09:51:57 GMT
Content-Type: text/html; charset=UTF-8
Transfer-Encoding: chunked
Connection: keep-alive
Vary: HTTPS
X-Powered-By: PHP/7.4.28
Expires: Fri, 07 Jun 1974 04:00:00 GMT
Last-Modified: Sun, 12 Feb 2023 09:19:02 GMT
X-Bitrix-Composite: Cache (200)
X-Content-Type-Options: nosniff
X-Frame-Options: SAMEORIGIN
Referrer-Policy: strict-origin-when-cross-origin
Strict-Transport-Security: max-age=15768000
Set-Cookie: SESSID=bbce; path=/; Secure; HttpOnly
Content-Encoding: gzip

[('и', 44), (',', 43), ('в', 22), ('иннополис', 20), ('.', 19), ('с', 13), ('на', 12), ('университет', 11), ('университета', 11), ('центр', 10)]


## 1.4. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [9]:
from queue import Queue

class Crawler:
    
    def crawl_generator(self, source, depth=1):
      #f we are required to process a file once, 
      #we can add a set for tracking the urls but
      # since it wasn't a requirement, we skip that
        q = Queue()
        q.put((source,0))      
        while True:
          try:
              url, index = q.get_nowait()
              doc = HtmlDocumentTextData(url)
              if hasattr(doc.doc, 'anchors') is False:
                continue

              for val, url in doc.doc.anchors:
                if(index+1 <= depth):
                  q.put((url, index+1))
              # map(q.put, list(map(lambda val: (val,index+1), doc.doc.anchors)))
              yield doc
          except Exception as e:
              print(url, e)
              break

### 1.4.1. Tests ###

In [10]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

Streaming output truncated to the last 5000 lines.
33786 distinct word(s) so far
File Not Found
Server: nginx/1.18.0 (Ubuntu)
Date: Sun, 12 Feb 2023 10:39:37 GMT
Content-Type: text/html; charset=UTF-8
Transfer-Encoding: chunked
Connection: keep-alive
Vary: HTTPS
P3P: policyref="/bitrix/p3p.xml", CP="NON DSP COR CUR ADM DEV PSA PSD OUR UNR BUS UNI COM NAV INT DEM STA"
X-Powered-CMS: Bitrix Site Manager (8a5fc641d5a15a45d4005bf699c31dee)
Set-Cookie: PHPSESSID=JF7LxetBM0bT0Qey7qvjFd0M9LYfF4Wi; path=/; domain=minobrnauki.gov.ru; HttpOnly, BITRIX_SM_GUEST_ID=16945330; expires=Wed, 07-Feb-2024 10:40:10 GMT; Max-Age=31104000; path=/; domain=minobrnauki.gov.ru, BITRIX_SM_LAST_VISIT=12.02.2023%2013%3A40%3A10; expires=Wed, 07-Feb-2024 10:40:10 GMT; Max-Age=31104000; path=/; domain=minobrnauki.gov.ru
Expires: Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control: no-store, no-cache, must-revalidate
Pragma: no-cache
X-Frame-Options: SAMEORIGIN, SAMEORIGIN
X-Content-Type-Options: nosniff
Content-Encoding: gz